In [1]:
import os
import pandas as pd
from neuralprophet import NeuralProphet
from neuralprophet import set_random_seed
from dotenv import load_dotenv
import dropbox
import requests
from utils import upload


load_dotenv()

# Function to reformat the forecast data


def reformat_forecast(forecast):
    forecast_data = []

    for i in range(8):
        step_col = f"step{i}"
        quantile_20_col = f"step{i} 20.0%"  # Column name for 20th percentile
        quantile_80_col = f"step{i} 80.0%"  # Column name for 80th percentile

        forecast_data.append({
            "ds": forecast["ds"].iloc[0] + pd.Timedelta(hours=i),
            "yhat": forecast[step_col].iloc[0],
            # Add lower bound (20th percentile)
            "yhat_lower": forecast[quantile_20_col].iloc[0],
            # Add upper bound (80th percentile)
            "yhat_upper": forecast[quantile_80_col].iloc[0]
        })

    new_forecast_df = pd.DataFrame(forecast_data)
    return new_forecast_df


data = pd.read_csv(
    'https://www.dropbox.com/scl/fi/ksf0nbmmiort5khbrgr61/allData.csv?rlkey=75e735fjk4ifttjt553ukxt3k&dl=1')
data.ds = pd.to_datetime(data.ds)
data = data.sort_values('ds')
data['y'] = data['POD_GREEN_TBS']+data['POD_YELLOW_TBS']+data['POD_ORANGE_TBS'] + \
    data['TRG_HALLWAY_TBS']+data['RAZ_TBS'] + \
    data['AMBVERTTBS']+data['QTrack_TBS']+data['Garage_TBS']
df = data.copy()
df = df[['ds', 'y']]



Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


In [2]:
df

,ds,y
0,2021-01-01 01:00:00,0.0
1,2021-01-01 02:00:00,3.0
2,2021-01-01 03:00:00,2.0
3,2021-01-01 04:00:00,1.0
4,2021-01-01 05:00:00,2.0
...,...,...
36064,2025-02-14 10:00:00,5.0
36065,2025-02-14 11:00:00,11.0
36066,2025-02-14 12:00:00,13.0
36067,2025-02-14 13:00:00,23.0


In [18]:
regressors = [
#  'ds',
 'INFLOW_STRETCHER',
 'Infl_Stretcher_cum',
 'INFLOW_AMBULATORY',
 'Infl_Ambulatory_cum',
 'Inflow_Total',
 'Inflow_Cum_Total',
 'INFLOW_AMBULANCES',
 'Infl_Ambulances_cum',
 'FLS',
 'CUM_ADMREQ',
 'CUM_BA1',
 'WAITINGADM',
 'TTStr',
 'TRG_HALLWAY1',
 'TRG_HALLWAY_TBS',
 'reoriented_cum',
 'reoriented_cum_MD',
 'QTRACK1',
 'RESUS',
 'Pod_T',
 'POD_GREEN',
 'POD_GREEN_TBS',
 'POD_YELLOW',
 'POD_YELLOW_TBS',
 'POD_ORANGE',
 'POD_ORANGE_TBS',
 'POD_CONS_MORE2H',
 'POD_IMCONS_MORE4H',
 'POD_XRAY_MORE2H',
 'POD_CT_MORE2H',
 'POST_POD1',
 'VERTSTRET',
 'RAZ_TBS',
 'RAZ_LAZYBOY',
 'RAZ_WAITINGREZ',
 'AMBVERT1',
 'AMBVERTTBS',
 'QTrack_TBS',
 'Garage_TBS',
 'RAZ_CONS_MORE2H',
 'RAZ_IMCONS_MORE4H',
 'RAZ_XRAY_MORE2H',
 'RAZ_CT_MORE2H1',
 'PSYCH1',
 'PSYCH_WAITINGADM',
#  'y'
 ]

In [19]:
for regressor in regressors:
    df[regressor] = data[regressor] 

In [20]:
df

,ds,y,INFLOW_STRETCHER,Infl_Stretcher_cum,INFLOW_AMBULATORY,Infl_Ambulatory_cum,Inflow_Total,Inflow_Cum_Total,INFLOW_AMBULANCES,Infl_Ambulances_cum,...,AMBVERT1,AMBVERTTBS,QTrack_TBS,Garage_TBS,RAZ_CONS_MORE2H,RAZ_IMCONS_MORE4H,RAZ_XRAY_MORE2H,RAZ_CT_MORE2H1,PSYCH1,PSYCH_WAITINGADM
0,2021-01-01 01:00:00,0.0,1,1,1,1,2,2,0,0,...,0,0,0.0,0.0,1,0,0,0,3,3
1,2021-01-01 02:00:00,3.0,2,3,1,2,3,5,1,1,...,2,2,0.0,0.0,1,1,0,0,3,3
2,2021-01-01 03:00:00,2.0,0,3,1,3,1,6,0,1,...,2,1,0.0,0.0,0,0,0,0,3,3
3,2021-01-01 04:00:00,1.0,1,4,0,3,1,7,1,2,...,2,0,0.0,0.0,0,0,0,0,4,3
4,2021-01-01 05:00:00,2.0,1,5,1,4,2,9,1,3,...,1,1,0.0,0.0,0,0,0,0,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36053,2025-02-13 17:00:00,23.0,5,107,4,44,9,151,1,25,...,12,10,2.0,0.0,4,0,2,0,13,8
36054,2025-02-13 18:00:00,24.0,10,117,3,47,13,164,4,29,...,15,13,0.0,0.0,6,0,1,0,13,9
36055,2025-02-13 19:00:00,18.0,8,125,1,48,9,173,3,32,...,10,10,1.0,0.0,7,0,0,0,10,7
36056,2025-02-13 20:00:00,10.0,3,128,1,49,4,177,3,35,...,6,6,0.0,0.0,7,0,0,3,11,8


In [21]:

params = {
    'growth': 'off',
    'yearly_seasonality': True,
    'weekly_seasonality': True,
    'daily_seasonality': True,
    'n_lags': 48,
    'n_forecasts': 8,
    'epochs': 10,
    'quantiles': [0.2, 0.5, 0.8]
}
for regressor in regressors:
    # print(regressor)
    m = NeuralProphet(**params)
    m.add_lagged_regressor(regressor)
    set_random_seed(42)
    metrics = m.fit(df[['ds','y',regressor]], freq='h')
    print(regressor,metrics)



INFO - (NP.forecaster.add_lagged_regressor) - n_lags = 'auto', number of lags for regressor is set to Autoregression number of lags (48)
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (py.warnings._showwarnmsg) - /Users/devinhopkins/anaconda3/envs/ed-flow-2023/lib/python3.10/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency h corresponds to 99.895% of the data.
WARNING - (py.warnings._showwarnmsg) - /Users/devinhopkins/anaconda3/envs/ed-flow-2023/lib/python3.10/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  conve

Training: |          | 0/? [00:00<?, ?it/s]

Finding best initial lr:   0%|          | 0/263 [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

INFO - (NP.forecaster.add_lagged_regressor) - n_lags = 'auto', number of lags for regressor is set to Autoregression number of lags (48)
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (py.warnings._showwarnmsg) - /Users/devinhopkins/anaconda3/envs/ed-flow-2023/lib/python3.10/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency h corresponds to 99.895% of the data.
WARNING - (py.warnings._showwarnmsg) - /Users/devinhopkins/anaconda3/envs/ed-flow-2023/lib/python3.10/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  conve

...

INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 128


Training: |          | 0/? [00:00<?, ?it/s]

Finding best initial lr:   0%|          | 0/263 [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

PSYCH_WAITINGADM    train_loss  reg_loss        MAE       RMSE      Loss  RegLoss  epoch
0    0.905603       0.0  21.563644  27.717295  0.905020      0.0      0
1    0.188008       0.0   8.197437  10.430122  0.187945      0.0      1
2    0.098691       0.0   6.601185   8.451581  0.098682      0.0      2
3    0.080759       0.0   6.110296   7.856183  0.080750      0.0      3
4    0.073400       0.0   5.881519   7.585192  0.073397      0.0      4
5    0.069713       0.0   5.779950   7.463051  0.069716      0.0      5
6    0.068491       0.0   5.746957   7.424028  0.068484      0.0      6
7    0.068000       0.0   5.725520   7.400734  0.067999      0.0      7
8    0.067472       0.0   5.710702   7.381945  0.067464      0.0      8
9    0.067225       0.0   5.704830   7.373335  0.067223      0.0      9


In [15]:

df_future = m.make_future_dataframe(df, periods=8)
forecast = m.predict(df_future, decompose=False, raw=True)

output_df = reformat_forecast(forecast)

output_df.to_csv('total_tbs_np.csv', index=False)

dropbox_app_key = os.environ.get("DROPBOX_APP_KEY")
dropbox_app_secret = os.environ.get("DROPBOX_APP_SECRET")
dropbox_refresh_token = os.environ.get("DROPBOX_REFRESH_TOKEN")

# exchange the authorization code for an access token:
token_url = "https://api.dropboxapi.com/oauth2/token"
params = {
    "grant_type": "refresh_token",
    "refresh_token": dropbox_refresh_token,
    "client_id": dropbox_app_key,
    "client_secret": dropbox_app_secret
}
r = requests.post(token_url, data=params)
# print(r.text)

dropbox_access_token = r.json()['access_token']

dbx = dropbox.Dropbox(dropbox_access_token)

upload(dbx, 'total_tbs_np.csv', '', '',
            'total_tbs_np.csv', overwrite=True)

print(output_df)

WARNING - (py.warnings._showwarnmsg) - /Users/devinhopkins/anaconda3/envs/ed-flow-2023/lib/python3.10/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency h corresponds to 99.895% of the data.
WARNING - (py.warnings._showwarnmsg) - /Users/devinhopkins/anaconda3/envs/ed-flow-2023/lib/python3.10/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

WARNING - (py.warnings._showwarnmsg) - /Users/devinhopkins/anaconda3/envs/ed-flow-2023/lib/python3.10/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and w

Predicting: |          | 0/? [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x2112 and 2160x24)

In [6]:

params = {
    'growth': 'off',
    'yearly_seasonality': True,
    'weekly_seasonality': True,
    'daily_seasonality': True,
    'n_lags': 48,
    'n_forecasts': 8,
    'epochs': 20,
    'quantiles': [0.2, 0.5, 0.8]
}
m = NeuralProphet(**params)
set_random_seed(42)
metrics = m.fit(df[['ds','y']], freq='h')
print(metrics)



WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (py.warnings._showwarnmsg) - /Users/devinhopkins/anaconda3/envs/ed-flow-2023/lib/python3.10/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency h corresponds to 99.892% of the data.
WARNING - (py.warnings._showwarnmsg) - /Users/devinhopkins/anaconda3/envs/ed-flow-2023/lib/python3.10/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

WARNING - (py.warnings._showwarnmsg) - /Users/devinhopkins/anaconda3/env

Training: |          | 0/? [00:00<?, ?it/s]

Finding best initial lr:   0%|          | 0/263 [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

    train_loss  reg_loss        MAE       RMSE      Loss  RegLoss  epoch
0     0.422548       0.0  11.587409  14.771971  0.422409      0.0      0
1     0.095270       0.0   6.513811   8.362106  0.095259      0.0      1
2     0.074441       0.0   5.842206   7.545600  0.074438      0.0      2
3     0.067161       0.0   5.570193   7.237640  0.067160      0.0      3
4     0.062264       0.0   5.406718   7.055072  0.062261      0.0      4
5     0.058913       0.0   5.287559   6.908538  0.058916      0.0      5
6     0.058202       0.0   5.262602   6.869322  0.058204      0.0      6
7     0.057825       0.0   5.262011   6.865791  0.057825      0.0      7
8     0.057176       0.0   5.241456   6.845469  0.057177      0.0      8
9     0.056325       0.0   5.224310   6.822224  0.056323      0.0      9
10    0.055562       0.0   5.208122   6.801084  0.055561      0.0     10
11    0.055096       0.0   5.195384   6.786657  0.055103      0.0     11
12    0.055049       0.0   5.194047   6.784097  0.0

In [7]:

df_future = m.make_future_dataframe(df, periods=8)
forecast = m.predict(df_future, decompose=False, raw=True)

output_df = reformat_forecast(forecast)

# output_df.to_csv('total_tbs_np.csv', index=False)

# dropbox_app_key = os.environ.get("DROPBOX_APP_KEY")
# dropbox_app_secret = os.environ.get("DROPBOX_APP_SECRET")
# dropbox_refresh_token = os.environ.get("DROPBOX_REFRESH_TOKEN")

# # exchange the authorization code for an access token:
# token_url = "https://api.dropboxapi.com/oauth2/token"
# params = {
#     "grant_type": "refresh_token",
#     "refresh_token": dropbox_refresh_token,
#     "client_id": dropbox_app_key,
#     "client_secret": dropbox_app_secret
# }
# r = requests.post(token_url, data=params)
# # print(r.text)

# dropbox_access_token = r.json()['access_token']

# dbx = dropbox.Dropbox(dropbox_access_token)

# upload(dbx, 'total_tbs_np.csv', '', '',
#             'total_tbs_np.csv', overwrite=True)

print(output_df)

WARNING - (py.warnings._showwarnmsg) - /Users/devinhopkins/anaconda3/envs/ed-flow-2023/lib/python3.10/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency h corresponds to 99.892% of the data.
WARNING - (py.warnings._showwarnmsg) - /Users/devinhopkins/anaconda3/envs/ed-flow-2023/lib/python3.10/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

WARNING - (py.warnings._showwarnmsg) - /Users/devinhopkins/anaconda3/envs/ed-flow-2023/lib/python3.10/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and w

Predicting: |          | 0/? [00:00<?, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


                   ds       yhat  yhat_lower  yhat_upper
0 2025-02-14 09:00:00  10.068016    7.630710   12.632497
1 2025-02-14 10:00:00  13.380965   10.698642   16.102781
2 2025-02-14 11:00:00  16.532358   13.059581   19.845911
3 2025-02-14 12:00:00  21.068089   17.599720   25.140837
4 2025-02-14 13:00:00  26.141987   21.976856   30.338787
5 2025-02-14 14:00:00  28.455517   24.572857   33.114147
6 2025-02-14 15:00:00  30.958015   26.626717   35.936363
7 2025-02-14 16:00:00  34.713673   30.409586   39.647045


In [8]:
df_future

,ds,y
0,2025-02-12 09:00:00,9.0
1,2025-02-12 10:00:00,20.0
2,2025-02-12 11:00:00,21.0
3,2025-02-12 12:00:00,27.0
4,2025-02-12 13:00:00,35.0
5,2025-02-12 14:00:00,42.0
6,2025-02-12 15:00:00,47.0
7,2025-02-12 16:00:00,52.0
8,2025-02-12 17:00:00,45.0
9,2025-02-12 18:00:00,45.0


In [9]:
forecast

,ds,step0,step0 20.0%,step0 80.0%,step1,step1 20.0%,step1 80.0%,step2,step2 20.0%,step2 80.0%,...,step4 80.0%,step5,step5 20.0%,step5 80.0%,step6,step6 20.0%,step6 80.0%,step7,step7 20.0%,step7 80.0%
0,2025-02-14 09:00:00,10.068016,7.630710,12.632497,13.380965,10.698642,16.102781,16.532358,13.059581,19.845911,...,30.338787,28.455517,24.572857,33.114147,30.958015,26.626717,35.936363,34.713673,30.409586,39.647045
1,2025-02-14 10:00:00,14.276589,11.649502,17.153530,17.600025,14.600632,21.445171,22.145241,18.831219,26.837095,...,34.190331,31.877697,27.187386,36.719158,35.540882,31.247871,40.535683,30.916977,26.498934,35.498569
2,2025-02-14 11:00:00,9.800261,6.738299,13.464458,14.874308,11.239412,18.650829,20.370113,15.903036,23.959936,...,31.357994,30.999384,26.994108,35.654846,27.161476,22.899399,31.369753,24.366755,20.121973,28.221754
3,2025-02-14 12:00:00,15.703953,11.895148,19.571209,21.123354,17.098917,25.214832,24.328043,20.728416,29.111145,...,36.097820,27.715290,23.440998,31.861280,24.877886,20.834827,29.021091,23.002672,19.061331,27.043165
4,2025-02-14 13:00:00,18.520569,14.399103,22.685526,22.038666,18.147249,26.577795,25.317553,20.924141,29.979158,...,30.575108,23.510010,19.356779,27.544096,21.858780,18.118147,25.772263,20.208477,16.490715,24.590559
5,2025-02-14 14:00:00,25.472080,21.456884,29.952667,28.568628,24.171028,32.978916,32.789524,28.682045,37.231213,...,29.822470,24.031328,20.056955,28.239107,22.092533,17.728489,26.654118,21.123177,16.734415,25.098757
6,2025-02-14 15:00:00,24.551563,20.214811,29.225599,29.126873,25.131115,33.868992,25.544039,21.185181,30.025583,...,25.600353,19.864237,15.646545,24.082916,19.239283,15.477976,23.505707,16.765371,12.938387,21.275837
